In [1]:
import os
import pickle
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import datetime
from dateutil.relativedelta import relativedelta

def get_last_day_of_the_quarter(dt):
    current_quarter = int((dt.month - 1) / 3 + 1)
    return datetime.date(dt.year, 3 * current_quarter, 1) + \
        relativedelta(months=1) - relativedelta(days=1)

def get_last_day_of_the_month(dt):
    return datetime.date(dt.year, dt.month, 1) + \
        relativedelta(months=1) - relativedelta(days=1)


In [3]:
files = glob.glob('**/raw_data.pkl')
files.sort(key=os.path.getmtime)

print(f'latest dataset: {files[-1]}')
dataset = pickle.load(open(files[-1], 'rb'))

latest dataset: output_2022-02-08T20:02:28/raw_data.pkl


In [9]:

def postprocess_time(dataset):
    result = {}
    for index, df in dataset.items():
        if 'time' not in index:
            result[index] = df
            continue
        if 'fund_id' in index:
            time_processor = get_last_day_of_the_quarter
        else:
            time_processor = get_last_day_of_the_month

        melt = df.reset_index().melt(id_vars=index).dropna()
        melt.time = melt.time.apply(time_processor)
        melt = melt.groupby(list(index)).last()

        result[index] = melt

    return result
    

In [ ]:
dataset_postprocessed = postprocess_time(dataset)